# Description
这里使用TensorFlow 1.5实现用于CTR预测的DeepFM，数据集选用的是kaggle上的criteo数据集。数据集的详细介绍可以参看[FM代码实现](https://github.com/Andr-Robot/ML_Model/blob/main/FM.ipynb)。

# DeepFM
DeepFM模型包含FM和DNN两部分，FM模型可以抽取low-order特征，DNN可以抽取high-order特征。相比于Wide & Deep模型无需人工特征工程。由于输入仅为原始特征，而且FM和DNN共享输入向量特征，DeepFM模型训练速度很快。    

![DeepFM架构图](https://raw.githubusercontent.com/Andr-Robot/iMarkdownPhotos/master/Res/ml/deepfm_architecture.png)